In [ ]:
import bs4
import os
from dotenv import load_dotenv
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 사용할 LLM 준비
## OpenAI

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
print(f"API key exists: {api_key is not None}")

# 웹페이지 불러오기

In [ ]:
loader = WebBaseLoader(
  web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
  bs_kwargs=dict(
    parse_only=bs4.SoupStrainer(
            class_=("css-j3idia", "editedContent")
        )
  ),
)

docs = loader.load()
print(f"로드된 문서 수: {len(docs)}")
print(f"첫 번째 문서 내용 일부: {docs[0].page_content[:100] if docs else 'No content'}")


In [ ]:
text_splitter = RecursiveCharacterTextSplitter( # 왓 이즈 디스 청크 차이가 잇나
    chunk_size=500,
    chunk_overlap=20
)

splits = text_splitter.split_documents(docs)
print(f"분할된 청크 수: {len(splits)}")

if len(splits) == 0:
    print("청크가 없어 원본 문서를 사용합니다.")
    splits = docs

In [ ]:
try:
    embeddings = OpenAIEmbeddings(api_key=api_key)
    # 간단한 테스트 임베딩 생성
    test_embedding = embeddings.embed_query("Hello world")
    print(f"테스트 임베딩 길이: {len(test_embedding)}")
    print("임베딩 테스트 성공!")
except Exception as e:
    print(f"임베딩 테스트 실패: {e}")

In [ ]:
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))
)
retriever = vectorstore.as_retriever()

In [ ]:
user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."
retrieved_docs = retriever.invoke(user_msg)

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
prompt = ChatPromptTemplate.from_template("너는 이 공모전을 평가했던 심사위원이이야. 한국어로 대답하고, 다음에 이 공모전을 참가할 사람들이 참고할 수 있도록 유익한 정보들을 선택해줄 수있어야해. 정보를 찾은 후 대답을 정리할 때는 너의 의견이 들어가면 안돼. 그리고 내가 제공한 문서에서 질문에 대한 대답을 찾을 수 없다면, 모른다고 대답해야해. <context>: {context} <question>: {question}")

In [ ]:
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(prompt)

In [ ]:
response = llm.invoke(user_prompt)
print(response.content)

In [ ]:
retrieved_docs